In [36]:
#standard environment setup
import pandas as pd
import numpy as np

#datetime imports for dates
import datetime as dt
import pytz

#ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [37]:
df=pd.read_csv("store_database.csv")
df.head()

,Unnamed: 0,item,sale_amount,sale_date,sale_id,store,store_address,store_city,store_id,store_state,store_zipcode,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253,Riceland,1.0,Riceland American Jazmine Rice,0.84,3.520026e+10,3.520026e+10
1,1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253,Riceland,1.0,Riceland American Jazmine Rice,0.84,3.520026e+10,3.520026e+10
2,2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253,Riceland,1.0,Riceland American Jazmine Rice,0.84,3.520026e+10,3.520026e+10
3,3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,1,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253,Riceland,1.0,Riceland American Jazmine Rice,0.84,3.520026e+10,3.520026e+10
4,4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,1,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253,Riceland,1.0,Riceland American Jazmine Rice,0.84,3.520026e+10,3.520026e+10


In [38]:
#1) Write a function to convert a date to a `datetime` data type.
def convert_to_datetime(col):
   return pd.to_datetime(col)

df.sale_date=convert_to_datetime(df.sale_date)
df.sale_date.dtype

datetime64[ns, UTC]

In [39]:
#2) Write a function to change a datetime to UTC
def convert_to_utc(col):
    return col.dt.tz_convert('UTC')
df.sale_date=convert_to_utc(df.sale_date)

In [40]:
#3)Write a function to parse a date column into 6 additional columns(while keeping the original date)
#:year,quarter,month,day of month,day of week,weekend vs. weekday

def parse_date(df,col):
    return df.assign(year=df[col].dt.year, quarter=df[col].dt.quarter,
          month=df[col].dt.month, day_of_week=df[col].dt.dayofweek,
          day_of_month=df[col].dt.day,
          weekend_or_weekday=lambda df: (df.day_of_week == 0) | (df.day_of_week == 6))
df=parse_date(df,'sale_date')

In [41]:
#4) Add a column to your dataframe called `sales_total`,which is derived from sale_amount(total items) and item price
def add_sales_total(df):
    df['sales_total']=df.sale_amount*df.item_price
    return df
df=add_sales_total(df)
df.head()

,Unnamed: 0,item,sale_amount,sale_date,sale_id,store,store_address,store_city,store_id,store_state,...,item_price,item_upc12,item_upc14,year,quarter,month,day_of_week,day_of_month,weekend_or_weekday,sales_total
0,0,1,13.0,2013-01-01 00:00:00+00:00,1,1,12125 Alamo Ranch Pkwy,San Antonio,1,TX,...,0.84,3.520026e+10,3.520026e+10,2013,1,1,1,1,False,10.92
1,1,1,11.0,2013-01-02 00:00:00+00:00,2,1,12125 Alamo Ranch Pkwy,San Antonio,1,TX,...,0.84,3.520026e+10,3.520026e+10,2013,1,1,2,2,False,9.24
2,2,1,14.0,2013-01-03 00:00:00+00:00,3,1,12125 Alamo Ranch Pkwy,San Antonio,1,TX,...,0.84,3.520026e+10,3.520026e+10,2013,1,1,3,3,False,11.76
3,3,1,13.0,2013-01-04 00:00:00+00:00,4,1,12125 Alamo Ranch Pkwy,San Antonio,1,TX,...,0.84,3.520026e+10,3.520026e+10,2013,1,1,4,4,False,10.92
4,4,1,10.0,2013-01-05 00:00:00+00:00,5,1,12125 Alamo Ranch Pkwy,San Antonio,1,TX,...,0.84,3.520026e+10,3.520026e+10,2013,1,1,5,5,False,8.40


In [42]:
#5) Create a new dataframe that aggregates the sales_total and sale_amount(item count) using sum and median
#by day of week
df_dow=df.groupby('day_of_week')['sales_total','sale_amount'].agg(['median','sum'])
df_dow

sales_total              sale_amount           
                 median          sum      median        sum
day_of_week                                                
0                162.98  11363125.56        38.0  5360807.0
1                190.20  13285070.79        44.0  6264707.0
2                191.08  13315908.37        44.0  6282865.0
3                202.88  14248570.71        47.0  6718573.0
4                218.73  15171556.09        50.0  7164546.0
5                231.87  16150420.13        53.0  7620039.0
6                247.28  17100584.41        56.0  8072103.0

In [43]:
#6) Explore the pandas Dataframe.diff() function.
#Create a new column that is the result of the current sales- the previous days sales.
df['diff_in_sales']=df['sales_total'].diff(periods=1)

In [44]:
#7) Write a function to set the index to be the datetime variable
def change_index(df,col):
    df=df.set_index(col)
    return df
df=change_index(df,'sale_date')
df.head()

,Unnamed: 0,item,sale_amount,sale_id,store,store_address,store_city,store_id,store_state,store_zipcode,...,item_upc12,item_upc14,year,quarter,month,day_of_week,day_of_month,weekend_or_weekday,sales_total,diff_in_sales
sale_date,,,,,,,,,,,,,,,,,,,,,
2013-01-01 00:00:00+00:00,0,1,13.0,1,1,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253,...,3.520026e+10,3.520026e+10,2013,1,1,1,1,False,10.92,NaN
2013-01-02 00:00:00+00:00,1,1,11.0,2,1,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253,...,3.520026e+10,3.520026e+10,2013,1,1,2,2,False,9.24,-1.68
2013-01-03 00:00:00+00:00,2,1,14.0,3,1,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253,...,3.520026e+10,3.520026e+10,2013,1,1,3,3,False,11.76,2.52
2013-01-04 00:00:00+00:00,3,1,13.0,4,1,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253,...,3.520026e+10,3.520026e+10,2013,1,1,4,4,False,10.92,-0.84
2013-01-05 00:00:00+00:00,4,1,10.0,5,1,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253,...,3.520026e+10,3.520026e+10,2013,1,1,5,5,False,8.40,-2.52


In [45]:
#def prepare_df(df,col):
#    df.sale_date=convert_to_datetime(col)
#    df=parse_date(df,col)
#    df=change_index(df,col)
#    return df
#df1=prepare_df(df,df.sale_date)    

In [46]:
df.to_csv("prepared_store.csv")